In [2]:
# Адрес,
# Общая площадь кв.м,
# Общая площадь жилых помещений, 
# Кол-во этажей,
# Кол-во подъездов,
# Кол-во жилых помещений,
# Факт признания дома аварийным

from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent
import time
import re
import csv

HOST = 'https://www.reformagkh.ru/myhouse'

useragent = UserAgent()
options = webdriver.FirefoxOptions()
options.add_argument(f'user-agent={useragent.random}')
URL = "https://www.reformagkh.ru/myhouse?tid=2208161"

In [3]:
def get_html_page(url):
    try:
        driver = webdriver.Firefox(
            executable_path = '/home/alex/work/geckodriver',
            options=options)
        driver.get(url=url)
        time.sleep(10)
        html_page = driver.page_source
        return(html_page)
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()

In [4]:
def get_suffixes(html_page):
    soup = BeautifulSoup(html_page, 'html')
    a_class = soup.find_all('a')
    href=[]
    for suffix in a_class:
        suffix = suffix.get('href')
        if suffix is not None:
            href.append(suffix)
    feature_suffixes = full_link([item for item in href if re.findall('tid+.\d+', item)])
    return(feature_suffixes)

In [5]:
def full_link(href_suffixes):
    feature_links = [HOST + suffix for suffix in href_suffixes]
    return(feature_links)

In [6]:
def get_max_pages(url):
    # Извлечение суффиксов домов в три этапа, 
    # 1 - получение первой страницы, 
    # 2 - получение количества доступных страниц
    # 3 - обход по всем страницам
    gorod = get_html_page(url)
    soup = BeautifulSoup(gorod, 'html')
    a_class = soup.find_all('a')
    number = []
    
    num_of_pages = soup.find_all('a', class_='page-link')
    for suffix in num_of_pages:
        suffix = suffix.get('data-page')
        if suffix is not None:
            number.append(suffix)
    max_pages = int(suffix) # последнее полученное число и есть максимальное число страниц
    return(max_pages)

In [7]:
def get_all_houses(url): 
    max_pag = get_max_pages(url)
    driver = webdriver.Firefox(
                executable_path = '/home/alex/work/geckodriver',
                options=options)
    driver.get(url)
    time.sleep(20)
    href = []
    
    for i in range(max_pag):
        driver.execute_script("window.open('about:blank', 'tab2')")
        driver.switch_to.window("tab2")
        driver.get(HOST + '?' + 'page=' + str(i+1) + '&limit=20&view=list&sort=name&order=asc')
        time.sleep(5)
        html_page = driver.page_source
        soup = BeautifulSoup(html_page,'html')
        a_class = soup.find_all('a')
        for suffix in a_class:
            suffix = suffix.get('href')
            if suffix is not None:
                suffix = 'https://www.reformagkh.ru' + suffix
                href.append(suffix)
    feature_suffixes = [item for item in href if re.findall('view+.\d+', item)]
    driver.close()
    driver.quit()
    return(feature_suffixes)

In [8]:
def get_house_data(house_url):
    a = []
    b = []
    c = []
    data = {'Адрес':'None',
            'Общая площадь, кв.м':'None',
            'Общая площадь жилых помещений, кв.м':'None',
            'Количество этажей, ед.':'None',
            'Количество подъездов, ед.':'None',
            'Количество жилых помещений':'None',
            'Количество лифтов, ед.': 'None',
            'Факт признания дома аварийным':'None',
            'Год ввода дома в эксплуатацию':'None',
            'Численность жителей, чел.':'None'}
    
    house_page = get_html_page(house_url)
    soup = BeautifulSoup(house_page, 'html')
    address = soup.find_all('h2')
    house_address = address[0].get_text()
    house_address = house_address.replace(',',';')
    
    features = soup.find_all('div', class_='house-specs d-flex fw-600 f-14 lh-19 align-items-start mb-3')
    house_data = [item.get_text() for item in features]
    
    for i in house_data:
        a.append(i.replace(',','.').replace(' ',''))
    for iter in a:
        nums = re.findall(r'\d*\.\d+|\d+', iter)
        nums = [float(i) for i in nums]
        b = b + nums
    for i in house_data:    
        m = re.search('\n(.+?)\n', i)
        if m:
            found = m.group(1)
        c.append(found)
    
    dictionary = dict(zip(c, b))
    dictionary["Адрес"] = house_address
    data.update(dictionary)
    return(data)

In [1]:
def save_file(house_data_list):
    with open('houses.csv', 'w', newline = '') as file:
        writer = csv.writer(file, delimiter = ';')
        writer.writerow(["Адрес", 
                         "Общая площадь кв.м", 
                         "Общая площадь жилых помещений кв.м", 
                         "Количество этажей",
                         "Количество подъездов", 
                         "Количество жилых помещений",
                         "Количество лифтов",
                         "Факт признания дома аварийным",
                         "Год ввода дома в эксплуатацию",
                         "Численность жителей"])
        for item in house_data_list:
            writer.writerow([item["Адрес"],
                             item["Общая площадь, кв.м"],
                             item["Общая площадь жилых помещений, кв.м"],
                             item["Количество этажей, ед."],
                             item["Количество подъездов, ед."],
                             item["Количество жилых помещений"],
                             item["Количество лифтов, ед."],
                             item["Факт признания дома аварийным"],
                             item["Год ввода дома в эксплуатацию"],
                             item["Численность жителей, чел."]])

In [ ]:
def parse():
    i = 0
    house_datas = []
    all_selsovet_list = []
    main_page = get_html_page(URL)
    oblast_links = get_suffixes(main_page)
    for oblast in oblast_links:
            oblast_html = get_html_page(oblast)
            oblast_inside_links = get_suffixes(oblast_html)
            for rayon_or_gorod in oblast_inside_links:
                try:
                    rayon_html_page = get_html_page(rayon_or_gorod)
                    selsovet_links = get_suffixes(rayon_html_page)
                    for selsovet_link in selsovet_links:
                        try:
                            selsovet_house_links = get_all_houses(selsovet_link)
                            all_selsovet_list += selsovet_house_links
                            for rayon_house in all_selsovet_list:
                                rayon_house_dict = get_house_data(rayon_house)
                                house_datas.append(rayon_house_dict)
                        except:
                            i += 1
                except:
                    try:
                        gorod_houses_links = get_all_houses(rayon_or_gorod)
                        for gorod_house in gorod_houses_links:
                            gorod_house_dict = get_house_data(gorod_house)
                            house_datas.append(gorod_house_dict)
                    except:
                        i += 1
    save_file(house_datas)
    print("Число неправильно сформированных адресов " + str(i))

In [ ]:
parse()

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}]
[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Кол

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

...

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.

[{'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; с. Боровское; ул. Новая; д. 7', 'Общая площадь, кв.м': 417.0, 'Общая площадь жилых помещений, кв.м': 408.0, 'Количество этажей, ед.': 2.0, 'Количество подъездов, ед.': 2.0, 'Количество жилых помещений': 'None', 'Количество лифтов, ед.': 'None', 'Факт признания дома аварийным': 'None', 'Год ввода дома в эксплуатацию': 1964.0, 'Численность жителей, чел.': 12.0}, {'Адрес': 'край. Алтайский; р-н. Алейский; п. Заветы Ильича; ул. Школьная; д. 10', 'Общая площадь, кв.м': 418.2, 'Общая площадь жилых помещений, кв.м': 378.0, 'Количество этажей, ед.': 2.